# Pipes - Despliegue

- Cargamos el modelo
- Cargamos los datos futuros
- Aplicar tubería

In [8]:
#Cargamos librerías principales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
import pickle
filename = 'modeloCV.pkl'
modelTree,variables,labelencoder,min_max_scaler = pickle.load(open(filename, 'rb'))

In [10]:
print(labelencoder.classes_)

[0 1 2 3 4]


In [6]:
#Recorda documentar esto para poder ejecutar streamlit
#data = pd.read_csv("datos_futuros.csv")
#data.head()

,CULTIVO,'CICLO DE CULTIVO','GRUPO CULTIVO',SUBGRUPO,'Area Sembrada(ha)','AREA COSECHADA(ha)',PRODUCCION(t),RENDIMIENTO(t/ha),'NOMBRE CIENTIFICO CULTIVO'
0,'Aguacate demAs variedades',Permanente,Frutales,'Demas frutales',986.00,350.00,3500.00,10.00,'Persea americana'
1,'MaIz Tradicional',Transitorio,Cereales,Cereales,0.00,0.00,0.00,0.00,'Zea mays'
2,Frijol,Transitorio,Leguminosas,Leguminosas,12.00,6.00,6.00,1.00,'Phaseolus sp.'
3,'LimOn Pajarito',Permanente,Frutales,CItricos,3.99,3.99,39.85,9.99,'Citrus limon L.'
4,'MaIz Tradicional',Transitorio,Cereales,Cereales,5.50,5.00,8.39,1.68,'Zea mays'


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle

# --- CONFIGURACIÓN Y CARGA DE RECURSOS ---

def load_pipeline():
    try:
        filename = 'modeloCV.pkl'
        modelTree,variables,labelencoder,min_max_scaler = pickle.load(open(filename, 'rb'))
    except FileNotFoundError:
        st.error(f"Error: No se encontró el archivo del pipeline en '{filename}'.")
        st.stop()
    except Exception as e:
        st.error(f"Error al cargar el pipeline (pickle): {e}")
        st.stop()

# Cargamos los recursos (Modelo, lista de columnas de X_train, LabelEncoder, MinMaxScaler)

# --- DEFINICIÓN DE INTERFAZ GRÁFICA ---

st.title('Predicción de Estado Físico del Cultivo')
st.markdown("---")

# 1. Inputs Numéricos
areaSembrada = st.slider('Area Sembrada(ha)', min_value=0, max_value=1000, value=20, step=1)
areaCosechada = st.slider('AREA COSECHADA(ha)', min_value=0, max_value=350, value=20, step=1)
produccion = st.slider('PRODUCCION(t)', min_value=0, max_value=3500, value=20, step=1)
rendimiento = st.slider('RENDIMIENTO(t/ha)', min_value=0.0, max_value=10.0, value=5.0, step=0.1)

# 2. Inputs Categóricos
cultivo = st.selectbox('CULTIVO', ["Aguacate demAs variedades", "MaIz Tradicional", "Frijol", "'LimOn Pajarito"])
cicloCultivo = st.selectbox('CICLO CULTIVO', ["Permanente", "Transitorio"])
grupoCultivo = st.selectbox('GRUPO CULTIVO', ["Frutales", "Cereales","Leguminosas"])
subGrupo = st.selectbox('SUBGRUPO', ["Demas frutales", "Cereales","Leguminosas", "CItricos"])
nombreCientifico = st.selectbox('NOMBRE CIENTIFICO CULTIVO', ["Persea americana", "Zea mays","Phaseolus sp", "Citrus limon L."])

# --- LÓGICA DE PREDICCIÓN CON BOTÓN ---

if st.button('Predecir Estado Físico del Cultivo'):

    # 3. Crear DataFrame con los datos capturados (¡CORRECCIÓN APLICADA AQUÍ!)
    datos = [[areaSembrada, areaCosechada, produccion, rendimiento, cultivo, cicloCultivo, grupoCultivo, subGrupo, nombreCientifico]]

    # MODIFICACIÓN: Se añaden comillas simples a las columnas numéricas donde el scaler las espera
    input_cols_corrected = [
        "'Area Sembrada(ha)'",
        "'AREA COSECHADA(ha)'",
        'PRODUCCION(t)',
        'RENDIMIENTO(t/ha)',
        'CULTIVO',
        'CICLO CULTIVO',
        'GRUPO CULTIVO',
        'SUBGRUPO',
        'NOMBRE CIENTIFICO CULTIVO'
    ]
    data_input = pd.DataFrame(datos, columns=input_cols_corrected)

    # 4. Pre-procesamiento (OHE y Reindex)
    try:
        # A. Definir columnas (¡CORRECCIÓN APLICADA AQUÍ!)
        # MODIFICACIÓN: num_cols debe tener las comillas
        num_cols = [
            "'Area Sembrada(ha)'",
            "'AREA COSECHADA(ha)'",
            'PRODUCCION(t)',
            'RENDIMIENTO(t/ha)'
        ]
        cat_cols = ['CULTIVO', 'CICLO CULTIVO', 'GRUPO CULTIVO', 'SUBGRUPO', 'NOMBRE CIENTIFICO CULTIVO']

        data_num = data_input[num_cols] # Ahora selecciona las columnas con comillas
        data_cat = data_input[cat_cols]

        # B. Aplicar el MinMaxScaler guardado a las columnas numéricas
        # ¡Esta línea ya no dará error de nombres!
        data_num_scaled = min_max_scaler.transform(data_num)
        data_num_scaled = pd.DataFrame(data_num_scaled, columns=num_cols, index=data_input.index)

        # C. Aplicación explícita de Dummies (OHE) y el resto de la lógica...
        data_cat_ohe = pd.get_dummies(data_cat, columns=cat_cols, drop_first=False)

        # D. Combinación y Alineación
        data_preparada = pd.concat([data_num_scaled, data_cat_ohe], axis=1)
        data_preparada = data_preparada.reindex(columns=variables, fill_value=0)

        # ... (Resto de la lógica de predicción y display)
        pred_encoded = modelTree.predict(data_preparada)
        pred_decoded = labelencoder.inverse_transform(pred_encoded)

        st.success('✅ Predicción Exitosa')
        st.metric(
            label="Estado Físico Predicho",
            value=f"El cultivo se encuentra en estado: **{pred_decoded}**"
        )

    except Exception as e:
        # El error NameError: name 'X_predict' is not defined se solucionará
        # porque X_predict se definirá si el ValueError se resuelve.
        st.error(f"Error al realizar la predicción: {e}")
        # Muestra el DataFrame antes de la predicción si falla.
        # st.dataframe(X_predict)

In [ ]:
#Se realiza la preparación debe ser igual al aprendizaje
#data_preparada=data.copy()
#data_preparada = pd.get_dummies(data, columns=["'CICLO DE CULTIVO'"], drop_first=True, dtype=int)
#data_preparada = pd.get_dummies(data, columns=['CULTIVO', "'GRUPO CULTIVO'", 'SUBGRUPO', "'NOMBRE CIENTIFICO CULTIVO'"], drop_first=False, dtype=int)

#data_preparada.head()

In [ ]:
#Se adicionan las columnas faltantes
#data_preparada=data_preparada.reindex(columns=variables,fill_value=0)
#data_preparada.head()

# **Predicciones**

In [ ]:
#Hacemos la predicción con el Tree
#Y_fut = modelTree.predict(data_preparada)
#print(Y_fut)

In [ ]:
#print(labelencoder.inverse_transform(Y_fut))

In [ ]:
#data['Tree']=labelencoder.inverse_transform(Y_fut)
#data.head()

In [ ]:
#Hacemos la predicción con Ranfom Forest
#Y_fut = model_rf.predict(data_preparada)
#data['RF']=labelencoder.inverse_transform(Y_fut)
#data.head()